In [5]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"


def make_tax(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=25)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_household(name="household", members=members, state_code="MO")
    sim.vary("employment_income", max=100_000, step=100)
    employment_income = sim.calc("employment_income")[0]
    mo_income_tax = sim.calc("mo_income_tax")[0].round()
    mtr = sim.deriv("mo_income_tax", "employment_income", wrt_target="head")
    return pd.DataFrame(
        dict(
            employment_income=employment_income,
            mo_income_tax=mo_income_tax,
            mtr=mtr,
            adults=adults,
            children=str(children),
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_tax(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    mo_income_tax="Missouri income tax",
    mtr="Marginal tax rate",
    adults="Adults",
    children="Children",
)

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}

fig = px.line(
    df,
    "employment_income",
    "mo_income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Missouri income tax",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

ValueError: operands could not be broadcast together with shapes (1001,) (2002,) 